# Asymmetry Detection Models
This notebook shows the process of load and train of a LSTM neuronal network to classify the asymmetry laterality of a PD patient or a control. In this notebook
we will show different implementations of LSTMs (Normal and bidirectional) and with different hyperparameters.

## Requiered Imports

In [1]:
import pickle

import tensorflow as tf
import numpy as np

from keras_tuner.tuners import RandomSearch


from src.model.asa_models import build_basic_lstm, build_2layer_lstm

from src.settings import ROOT_DIR
from src.utils.data_split import get_features_target
from src.model.callbacks_builder import get_callbacks, build_tensorboard

## Data load

In [2]:
train_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'train.pkl'
val_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'val.pkl'

with open(train_data_path, 'rb') as file:
    train_data = pickle.load(file)

with open(val_data_path, 'rb') as file:
    val_data = pickle.load(file)

In [3]:
len(train_data)

124

In [4]:
X_train, y_train = get_features_target(train_data)
X_val, y_val = get_features_target(val_data)

In [5]:
max_length = max(len(x) for x in X_train)
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post', dtype='float32')

max_length = max(len(x) for x in X_val)
X_val_padded = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen=max_length, padding='post', dtype='float32')

In [6]:
y_train = np.array(y_train)
y_val = np.array(y_val)

## Tensor Board
This tool will help us to supervise the experimentation process of all the experimental models described in this notebook

In [7]:
%load_ext tensorboard
%tensorboard --logdir ../results/asa/lightning_logs/

Reusing TensorBoard on port 6006 (pid 17168), started 3 days, 20:02:44 ago. (Use '!kill 17168' to kill it.)

In [8]:
tensorboard_callback = build_tensorboard('asa')

## Models
Every model will be trained looking for the best hyperparameters, for this we will use Random Search.

In [9]:
RS_EPOCHS = 20

### Simple LSTM
This model is a 64-lstm with any hidden layers and just one dense layer to infer the output.
Optimizer: *Adam*
Loss: *binary_crossentropy*

In [10]:
BATCH_SIZE = 14
EPOCHS = 200

In [11]:
tunner_model = RandomSearch(
    build_basic_lstm,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1
)

In [18]:
tunner_model.search(X_train_padded, y_train,
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded, y_val)
                    )

Trial 4 Complete [00h 00m 48s]
val_accuracy: 0.7200000286102295

Best val_accuracy So Far: 0.7200000286102295
Total elapsed time: 00h 04m 24s


In [19]:
best_hyperparams = tunner_model.get_best_hyperparameters(1)[0]
model = tunner_model.hypermodel.build(best_hyperparams)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               78336     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 78465 (306.50 KB)
Trainable params: 78465 (306.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
checkpoint_model, early_stop_model = get_callbacks('asa', '64-lstm-no-hidden')

In [23]:
# Train the model
history_model = model.fit(
    X_train_padded,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_padded, y_val),
    callbacks=[tensorboard_callback, checkpoint_model, early_stop_model])

Epoch 1/200
9/9 [==============================] - ETA: 0s - loss: 0.7059 - accuracy: 0.6694
Epoch 1: val_accuracy improved from -inf to 0.72000, saving model to /content/AIForIMUParkinsonData/results/asa/64-lstm-no-hidden/tmp/ckpt/checkpoint.model.keras
9/9 [==============================] - 8s 600ms/step - loss: 0.7059 - accuracy: 0.6694 - val_loss: 0.6303 - val_accuracy: 0.7200
Epoch 2/200
9/9 [==============================] - ETA: 0s - loss: 0.6702 - accuracy: 0.6694
Epoch 2: val_accuracy did not improve from 0.72000
9/9 [==============================] - 5s 555ms/step - loss: 0.6702 - accuracy: 0.6694 - val_loss: 0.6292 - val_accuracy: 0.7200
Epoch 3/200
9/9 [==============================] - ETA: 0s - loss: 0.6520 - accuracy: 0.6694
Epoch 3: val_accuracy did not improve from 0.72000
9/9 [==============================] - 5s 582ms/step - loss: 0.6520 - accuracy: 0.6694 - val_loss: 0.5975 - val_accuracy: 0.7200
Epoch 4/200
9/9 [==============================] - ETA: 0s - loss: 0.6

### 2 Hidden layers LSTM

In [ ]:
BATCH_SIZE = 14
EPOCHS = 150

In [ ]:
tunner_lstm = RandomSearch(
    build_2layer_lstm,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1
)

In [ ]:
tunner_lstm.search(X_train_padded, y_train,
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded, y_val)
                    )

In [ ]:
best_hyperparams_lstm = tunner_model.get_best_hyperparameters(1)[0]
lstm = tunner_model.hypermodel.build(best_hyperparams_lstm)

In [ ]:
best_hyperparams_lstm

In [0]:
lstm.summary()

In [0]:
checkpoint_lstm, early_stop_lstm = get_callbacks('asa', 'lstm-2hidden')

In [0]:
history_lstm = lstm.fit(x=X_train,
                        y=y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_val_padded, y_val),
                        callbacks=[tensorboard_callback, checkpoint_lstm, early_stop_lstm])